In [6]:
# Setup paths - Files are now local in team-submissions
# sys.path.append(os.path.dirname(os.path.abspath(__file__))) # Not needed if files are local

import labs_utils
import quantum_kernels
import labs_utils_cpu
import mts_cpu
import cudaq
import numpy as np

def run_step_a_demo():
    print("\n=== Step A: CPU Validation Demo ===\n")
    
    # 1. Quantum Circuit Execution
    N = 7
    print(f"Problem Size: N={N}")
    
    G2, G4 = quantum_kernels.get_interactions(N)
    print(f"Interactions: {len(G2)} (2-body), {len(G4)} (4-body)")
    
    # Parameters matches paper/notebook
    T = 1.0
    n_steps = 1
    dt = T / n_steps
    
    thetas = [labs_utils.compute_theta(step * dt, dt, T, N, G2, G4) for step in range(1, n_steps + 1)]
    print(f"Thetas computed: {thetas}")
    
    print("Running Quantum Circuit on CPU...")
    # By default, cudaq uses qpp-cpu if no GPU or if not specified.
    # We can explicitly set it if we want, but local execution usually defaults correct.
    
    shots = 100
    result = cudaq.sample(quantum_kernels.trotterized_circuit, N, G2, G4, n_steps, dt, T, thetas, shots_count=shots)
    
    print(f"Sampling complete. Top results:")
    
    # Process results into population
    quantum_population = []
    
    for bitstring, count in list(result.items())[:5]:
        seq = np.array([1 if b == '0' else -1 for b in bitstring])
        energy = labs_utils_cpu.calculate_labs_energy(seq)
        print(f"  {bitstring}: count={count}, Energy={energy}")
        
    # Create full population list for MTS
    for bitstring, count in result.items():
        seq = np.array([1 if b == '0' else -1 for b in bitstring])
        for _ in range(count):
            quantum_population.append(seq)
            
    # 2. Hybrid Workflow (MTS)
    print("\nRunning Classical MTS seeded with Quantum results...")
    pop_size = 20
    
    best_seq, best_energy, _, _ = mts_cpu.memetic_tabu_search_with_init(
        quantum_population, pop_size=pop_size, p_mutate=0.2, N_tabu=5, max_iterations=50, tabu_iterations=20
    )
    
    print(f"MTS Complete.")
    print(f"Best Logic Found: {best_seq}")
    print(f"Best Energy: {best_energy}")
    
    # Verify validity against known optimal for N=7 (E=3 for N=7 is not optimal, E=1 is optimal for N=7, wait. Let's check literature.)
    # N=7 optimal is 2. (Mertens 1996)
    # Wait, N=7 optimal is 1? 
    # Example 1: N=3 (E=1). N=4 (E=2). N=5 (E=2 or 1?). N=5 optimal is 1 ([1,1,1,-1,1] -> 1).
    # N=7: [1,1,1,-1,-1,1,-1] -> E=3? Let's check my tests.
    
    # The user notebook says N=7 optimal is 3?
    # Actually N=7 optimal is E=2 (Mertens 1996). 
    # Let's see if we find it.
    
    if best_energy <= 3.0:
        print("SUCCESS: Found low energy solution.")
    else:
        print("Note: Energy seems high, optimization parameters might need tuning for perfect optimal.")

if __name__ == "__main__":
    run_step_a_demo()



=== Step A: CPU Validation Demo ===

Problem Size: N=7
Interactions: 9 (2-body), 13 (4-body)
Thetas computed: [1.5815977036565938e-18]
Running Quantum Circuit on CPU...
Sampling complete. Top results:
  1111011: count=1, Energy=15.0
  1111001: count=1, Energy=7.0
  1110111: count=1, Energy=19.0
  1110010: count=2, Energy=3.0
  1101101: count=1, Energy=23.0

Running Classical MTS seeded with Quantum results...
MTS Complete.
Best Logic Found: [-1 -1 -1  1  1 -1  1]
Best Energy: 3.0
SUCCESS: Found low energy solution.
